# Get 3RWW Calibrated Radar Rainfall Data from the Teragon Rainfall API

> *This notebook is a work-in-progress*

*Get timeseries rainfall data for an area of interest within Allegheny County*

---

When radar estimates of rainfall are calibrated with actual rain gauge data, a highly accurate and valuable source of rainfall data can be calculated over large geographic areas. The result is called *Calibrated Radar Rainfall Data*, or *Gauge-Adjusted Radar Rainfall Data (GARRD)*

3 Rivers Wet Weather (3RWW), with support from [Vieux Associates](http://www.vieuxinc.com/), uses calibrated data from the NEXRAD radar located in Moon Township, PA with rain gauge measurements collected during the same time period and rain event for every square kilometer in Allegheny County. The resulting rainfall data is equivalent in accuracy to having 2,276 rain gauges placed across the County.

You can view and explore this data on 3RWW's calibrated radar rainfall data site at [www.3riverswetweather.org/municipalities/calibrated-radar-rainfall-data](http://www.3riverswetweather.org/municipalities/calibrated-radar-rainfall-data)

This notebook walks through how to programmatically access 3RWW's massive repository of high resolution spatiotemporal rainfall data for Allegheny County via the ***Teragon Rainfall Dataset API*** for an area of interest. The Teragon Rainfall Dataset API accesses a cache of the historic rainfall data that has been processed by Vieux Associates. It is the same data you'll find on 3RWW's [calibrated-radar-rainfall-data](http://www.3riverswetweather.org/municipalities/calibrated-radar-rainfall-data) website (which is also managed by Teragon).

Complete documentation for the Teragon Rainfall Dataset API is available at [3rww.github.io/api-docs](https://3rww.github.io/api-docs/?language=Python#teragon-rainfall-dataset-api-10).

## First: Notebook Setup

~~This assumes you've got things set up following the recommendations in the ***Getting Started*** notebook.~~

In [1]:
# imports from the Python standard library
import json #read and write JSON

We're going to use a few external Python packages to make our lives easier:

In [2]:
# imports from 3rd-party libraries
# Requests - HTTP requests for humans
import requests
# PETL - an Extract/Transform/Load toolbox
import petl as etl
# sortedcontainers provides a way to have sorted dictionaries (before Python 3.7)
from sortedcontainers import SortedDict
# Python DateUtil (parser) - a helper for reading timestamps
from dateutil.parser import parse
# ArcGIS API for Python - for accessing 3RWW's open reference datasets in ArcGIS Online
from arcgis.gis import GIS
from arcgis import geometry, GeoAccessor
# for displaying things from the ArcGIS Online in this Jupyter notebook
from IPython.display import display

## 1. The basics of getting calibrated radar rainfall data

Getting rainfall data programmatically is a mostly straightforward endeavor: it requires you to submit a HTTP request with parameters specifying locations of interest and a time range. It returns a `csv`-like plain-text response where time intervals are on the x-axis, locations are the y-axis, and values are rainfall amounts. Data quality metadata is included. 

Complete API documentation is available at [3rww.github.io/api-docs](https://3rww.github.io/api-docs/?language=Python#teragon-rainfall-dataset-api-10).

To demonstrate the basics of making a call to the API, we'll first use some pre-selected pixel values; we'll demonstrate how to get pixel locations from geodata later on, and then revisit submitting the request for specific locations.


### Assemble the request payload

We'll use the Python `requests` library to make our calls to the API.

The Hurricane Ivan rainfall event in 2004 will server as an example (2004-09-17 03:00 to 2004-09-18 00:00); the pixels used are pre-selected for this example (we'll get to identifying pixels for an area of interest in a bit).

The request payload for that event, as a Python `dictionary` looks like this:

In [3]:
data = {
    'startyear': 2004,
    'startmonth': 9,
    'startday': 17,
    'starthour': 3,
    'endyear': 2004,
    'endmonth': 9,
    'endday': 18,
    'endhour': 0,
    'interval': 'Hourly',
    'zerofill': 'yes',
    'pixels': '148,134;149,134;148,133;149,133'
}

The Teragon Rainfall Dataset API only accesspts `POST` requests. Using the Python `requests` library, then, we construct our call like this, using the `post` method:

In [4]:
response = requests.post(
    url="http://web.3riverswetweather.org/trp:API.pixel",
    data=data
)
# Note that the `data` argument for `request.post` is explicitly 
# used here: the API does not accept the request payload as 
# query string parameters (`params`), which is the default behavior 
# for the requests library.

That's it. The data is contained in the `response` variable. As mentioned earlier, the API returns a `csv`-like plain-text response where time intervals are on the x-axis, locations are the y-axis, and values are rainfall amounts. You can print the response:

In [5]:
print(response.text)

Timestamp,148-134,,149-134,,148-133,,149-133,
2004/09/17 03:00:00,0.0000,-,0.0000,-,0.0000,-,0.0000,-
2004/09/17 04:00:00,0.0040,-,0.0010,-,0.0050,-,0.0020,-
2004/09/17 05:00:00,0.0030,-,0.0030,-,0.0100,-,0.0030,-
2004/09/17 06:00:00,0.0430,-,0.0400,-,0.0450,-,0.0410,-
2004/09/17 07:00:00,0.1370,-,0.1389,-,0.1300,-,0.1430,-
2004/09/17 08:00:00,0.1090,-,0.1140,-,0.1040,-,0.1080,-
2004/09/17 09:00:00,0.2160,-,0.2200,-,0.1770,-,0.2089,-
2004/09/17 10:00:00,0.4260,-,0.3669,-,0.4610,-,0.4109,-
2004/09/17 11:00:00,0.4520,-,0.4100,-,0.4700,-,0.4560,-
2004/09/17 12:00:00,0.5590,-,0.6290,-,0.4390,-,0.4760,-
2004/09/17 13:00:00,1.4609,-,1.0550,-,0.9660,-,0.6470,-
2004/09/17 14:00:00,1.0889,-,1.1510,-,1.0340,-,1.0679,-
2004/09/17 15:00:00,0.7379,-,0.8900,-,0.6890,-,0.8200,-
2004/09/17 16:00:00,0.9260,-,0.8760,-,0.9860,-,0.8720,-
2004/09/17 17:00:00,0.3690,-,0.3609,-,0.3480,-,0.3409,-
2004/09/17 18:00:00,0.1948,-,0.1950,-,0.1669,-,0.1968,-
2004/09/17 19:00:00,0.1940,-,0.1810,-,0.1700,-,0.1740,-
20

## 2. Working with the data from the API

That raw response was a little hard to read, so we'll use the wonderful Python `PETL` library to get something human-readable (you might just as easily swap in the Python Pandas library to do this sort of thing).

In [6]:
table = etl.fromcsv(etl.MemorySource(response.text.encode()))
etl.vis.displayall(table)

Timestamp,148-134,,149-134,,148-133,,149-133,
2004/09/17 03:00:00,0.0000,-,0.0000,-,0.0000,-,0.0000,-
2004/09/17 04:00:00,0.0040,-,0.0010,-,0.0050,-,0.0020,-
2004/09/17 05:00:00,0.0030,-,0.0030,-,0.0100,-,0.0030,-
2004/09/17 06:00:00,0.0430,-,0.0400,-,0.0450,-,0.0410,-
2004/09/17 07:00:00,0.1370,-,0.1389,-,0.1300,-,0.1430,-
2004/09/17 08:00:00,0.1090,-,0.1140,-,0.1040,-,0.1080,-
2004/09/17 09:00:00,0.2160,-,0.2200,-,0.1770,-,0.2089,-
2004/09/17 10:00:00,0.4260,-,0.3669,-,0.4610,-,0.4109,-
2004/09/17 11:00:00,0.4520,-,0.4100,-,0.4700,-,0.4560,-
2004/09/17 12:00:00,0.5590,-,0.6290,-,0.4390,-,0.4760,-


That's better. Note that each pixel column has a column that follows it: the API response includes data quality metadata for every value if it exists. In this example, there isn't any data quality issues noted, thus the `-` following every value.

Once you've noted any data quality issues, you might consider removing those additional columns and clean things up to make working with the data a bit simpler, as follows:

In [7]:
def clean_up_response(response_text):
    """a helper function for cleaning up the API response, using PETL
    
    params:
    response_text = a Python `requests` library `response.text` object
    
    returns:
    a PETL table object
    """
    table = etl.fromcsv(etl.MemorySource(response_text.encode()))
    # get a list of the existing table header values
    h = list(etl.header(table))
    # we know every other column is a notes column, so we identify those
    xy_cols = zip(*[iter(h[1:])] * 2)
    # store some things:
    new_header = ['Timestamp']
    fields_to_cut = []
    # then we iterate over those columns and
    for each in xy_cols:
        # note the correct id, assembled from columns
        id_col, note_col = each[0], each[1]
        # assemble a new notes column name
        notes_col = "{0}-n".format(id_col)
        # add those to our new header (array)
        new_header.extend([id_col, notes_col])
        # track fields that we might want to remove
        fields_to_cut.append(notes_col)
        
    short_header = list(set(new_header).difference(set(fields_to_cut)))

    # transform the table
    table_cleaned = etl \
        .setheader(table, new_header) \
        .select('Timestamp', lambda v: v.upper() != 'TOTAL')  \
        .convert('Timestamp', lambda t: parse(t).isoformat())  \
        .replaceall('N/D', None)\
        .cutout(*tuple(fields_to_cut))\
        .convert(
            {h: float for h in short_header if h != 'Timestamp'}
        )

    return table_cleaned

In [8]:
table_cleaned = clean_up_response(response.text)

etl.vis.displayall(table_cleaned)

Timestamp,148-134,149-134,148-133,149-133
2004-09-17T03:00:00,0.0,0.0,0.0,0.0
2004-09-17T04:00:00,0.004,0.001,0.005,0.002
2004-09-17T05:00:00,0.003,0.003,0.01,0.003
2004-09-17T06:00:00,0.043,0.04,0.045,0.041
2004-09-17T07:00:00,0.137,0.1389,0.13,0.143
2004-09-17T08:00:00,0.109,0.114,0.104,0.108
2004-09-17T09:00:00,0.216,0.22,0.177,0.2089
2004-09-17T10:00:00,0.426,0.3669,0.461,0.4109
2004-09-17T11:00:00,0.452,0.41,0.47,0.456
2004-09-17T12:00:00,0.559,0.629,0.439,0.476


There it is. Export that to CSV with PETL like this:

```python
etl.tocsv(table_cleaned, "path/to/save/your/data.csv")
```

Now what if we want to work with this a key-value store? Try this:

In [9]:
data = SortedDict()
for row in etl.transpose(table_cleaned).dicts():
    inside = SortedDict()
    for d in row.items():
        if d[0] != 'Timestamp':
            if d[1]:
                v = float(d[1])
            else:
                v = d[1]
            inside[d[0]] = v
    data[row['Timestamp']] = inside
print(json.dumps(data, indent=2))

{
  "148-133": {
    "2004-09-17T03:00:00": 0.0,
    "2004-09-17T04:00:00": 0.005,
    "2004-09-17T05:00:00": 0.01,
    "2004-09-17T06:00:00": 0.045,
    "2004-09-17T07:00:00": 0.13,
    "2004-09-17T08:00:00": 0.104,
    "2004-09-17T09:00:00": 0.177,
    "2004-09-17T10:00:00": 0.461,
    "2004-09-17T11:00:00": 0.47,
    "2004-09-17T12:00:00": 0.439,
    "2004-09-17T13:00:00": 0.966,
    "2004-09-17T14:00:00": 1.034,
    "2004-09-17T15:00:00": 0.689,
    "2004-09-17T16:00:00": 0.986,
    "2004-09-17T17:00:00": 0.348,
    "2004-09-17T18:00:00": 0.1669,
    "2004-09-17T19:00:00": 0.17,
    "2004-09-17T20:00:00": 0.06,
    "2004-09-17T21:00:00": 0.067,
    "2004-09-17T22:00:00": 0.3959,
    "2004-09-17T23:00:00": 0.195
  },
  "148-134": {
    "2004-09-17T03:00:00": 0.0,
    "2004-09-17T04:00:00": 0.004,
    "2004-09-17T05:00:00": 0.003,
    "2004-09-17T06:00:00": 0.043,
    "2004-09-17T07:00:00": 0.137,
    "2004-09-17T08:00:00": 0.109,
    "2004-09-17T09:00:00": 0.216,
    "2004-09-17T10:

This provides a quick rainfall time-series per pixel.

---

> *EndNote: We've started codifying the above processes in a "wrapper API" available at http://3rww-rainfall-api.civicmapper.com/apidocs/, so you don't have to post-process the data like we just demonstrated. Check it out.*

## 3. Getting reference geodata

As we've seen above, 3RWW's Rainfall Data API is not spatial: it returns rainfall values for locations at points in time, but those locations are only represented by 'Pixel' IDs; it does not provide actual geometry or coordinates for those pixels.

The challenge in using that API comes in formulating the location for the request. Location is specified with a "pixel ID", which translates to a location on a 1-kilometer grid set over Allegheny County, PA. The pixel (or pixels) is a required parameter; finding and entering those raw values is somewhat tedious.

To do anything that is location specific with this data (e.g., query rainfall in a specific watershed), you'll want some geodata for reference.

### Vieux Pixel Polygons

A copy of the pixels used for all calibrated radar rainfall products (created by Vieux) are available on [3RWW's Open Data Portal](http://data-3rww.opendata.arcgis.com/) and 3RWW's regular ArcGIS Online site at:

* [data-3rww.opendata.arcgis.com/datasets/228b1584b89a45308ed4256c5bedd43d_1](https://data-3rww.opendata.arcgis.com/datasets/228b1584b89a45308ed4256c5bedd43d_1), and
* [3rww.maps.arcgis.com/home/item.html?id=228b1584b89a45308ed4256c5bedd43d](https://3rww.maps.arcgis.com/home/item.html?id=228b1584b89a45308ed4256c5bedd43d)

...respectively. We can retrieve it programmatically a couple of ways:

* with the [ArcGIS API for Python](https://developers.arcgis.com/python/); or
* by using the [Python Requests library](http://docs.python-requests.org/en/master/) to make a call directly to the Portal's ArcGIS REST API.

We'll show both ways below. 

#### Using the ArcGIS API for Python

In [10]:
# Establish a connection to your 3RWW's ArcGIS Online portal.
gis = GIS('https://3rww.maps.arcgis.com')

We can search for the feature layer by name:

In [11]:
search_results = gis.content.search('Gauge Adjusted Radar Rainfall Data')
for item in search_results:
    display(item)
garrd_item = search_results[0]

<Item title:"Gauge Adjusted Radar Rainfall Data Geometry" type:Feature Layer Collection owner:CM_3RWW>

Alternatively, we can use the item `id` to directly find the feature layer:

In [12]:
garrd_id = "228b1584b89a45308ed4256c5bedd43d"
garrd_item = gis.content.get(itemid=garrd_id)
garrd_item

<Item title:"Gauge Adjusted Radar Rainfall Data Geometry" type:Feature Layer Collection owner:CM_3RWW>

Either way gets us `gaard_item`: a feature layer *collection* item, which contains individual feature layers. This one (we know from clicking on the item above), has both points and polygons variants of the GARRD reference geometry. We're interested in the polygons (grid). Get that as follows:

In [13]:
garrd_item.layers

[<FeatureLayer url:"https://services6.arcgis.com/dMKWX9NPCcfmaZl3/arcgis/rest/services/garrd/FeatureServer/0">,
 <FeatureLayer url:"https://services6.arcgis.com/dMKWX9NPCcfmaZl3/arcgis/rest/services/garrd/FeatureServer/1">]

In [14]:
# it's the second item, index 1
garrd_grid = garrd_item.layers[1]
garrd_grid

<FeatureLayer url:"https://services6.arcgis.com/dMKWX9NPCcfmaZl3/arcgis/rest/services/garrd/FeatureServer/1">

Since we're in a notebook now, the ArcGIS API for Python lets you put that on a map:

In [15]:
m = gis.map('Pittsburgh')
m.add_layer(garrd_grid)
m

MapView(layout=Layout(height='400px', width='100%'))

Finally, we can turn that into a `geojson`-like Python dictionary.

In [16]:
q = garrd_grid.query(out_sr=4326)
garrd_grid_geojson = q.to_geojson

#### Using `requests`

This approach is a little more hands on, but works without fuss and without the overhead of the ArcGIS API for Python used above.

We need to get the service `url` from the item detail page on 3RWW's Open Data Portal, and then construct query parameters for the request as a Python dictionary.

In [17]:
# service URL - note how '/query' is at the end of the URL
service_url = 'https://services6.arcgis.com/dMKWX9NPCcfmaZl3/ArcGIS/rest/services/garrd/FeatureServer/1/query'
# query string parameters
params = {
    'where': '1=1', # Esri's convention for returning everything from the ArcGIS REST API
    'outFields': 'PIXEL', # only include the GARRD 'PIXEL' field
    'outSR': '4326', # project as WGS 1984
    'f': 'geojson' # return as geojson
}

In [18]:
# make the request
garrd_grid_geojson_response = requests.get(service_url, params=params)
garrd_grid_geojson_response

<Response [200]>

In [19]:
# this gets us the response as a geojson-like Python dictionary.
garrd_grid_geojson = garrd_grid_geojson_response.json()

That gets us a `geojson` object of all pixels as a python dictionary.

### Area of Interest Polygons

Next, we'll establish an area of interest using a polygon from an existing dataset: the Saw Mill Run watershed.

Allegheny County has a watershed dataset in ArcGIS Online, so we'll use that for this example. It's available here:

* http://openac-alcogis.opendata.arcgis.com/datasets/364f4c3613164f79a1d8c84aed6c03e0_0

(Note that you could swap this out for any online geodata service that provides polygons, and this will work)

#### With ArcGIS API for Python

In [20]:
# use the item ID from the link above to get the layer
watersheds_item = gis.content.get(itemid="364f4c3613164f79a1d8c84aed6c03e0")
watersheds_layer = watersheds_item.layers[0]
basin = watersheds_layer.query(where="DESCR like '%GIRTYS%'", out_sr=4326)

In [21]:
m2 = gis.map('Pittsburgh')
m2.add_layer(basin)
m2

MapView(layout=Layout(height='400px', width='100%'))

#### With `requests` library

In [22]:
# service URL - note how '/query' is at the end of the URL
service_url = 'https://services1.arcgis.com/vdNDkVykv9vEWFX4/arcgis/rest/services/Watersheds/FeatureServer/0/query'
# query string parameters
params = {
    'where': "DESCR like '%GIRTYS%'", # get GIRTYS RUN
    'outFields': ['DESCR', 'AREA'], # only include the GARRD 'PIXEL' field
    'outSR': '4326', # project as WGS 1984
    'f': 'geojson' # return as geojson
}
# make the request
watershed_geojson_response = requests.get(service_url, params=params)
watershed_geojson_response.json()

{'type': 'FeatureCollection',
 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-79.9836718140761, 40.528261685893],
      [-79.9831128027776, 40.5276949931352],
      [-79.9792972629182, 40.5238269298859],
      [-79.9769995055271, 40.521492175636],
      [-79.973578251441, 40.5209102203109],
      [-79.9725605696155, 40.5207370517712],
      [-79.9700962911061, 40.5183338013472],
      [-79.9689517807423, 40.5151409111637],
      [-79.9664453615379, 40.5130049117923],
      [-79.9659077359717, 40.5125468762804],
      [-79.9652968760073, 40.5111878080906],
      [-79.964480849007, 40.5093730819332],
      [-79.962474508686, 40.5068439546526],
      [-79.9608568059377, 40.5041088091013],
      [-79.9606046484331, 40.5014659652595],
      [-79.9603457394921, 40.4987537451699],
      [-79.9620446641611, 40.4939810168438],
      [-79.9628780245181, 40.4917436950863],
      [-79.9631023

> *Note that while we're pulling our data from online sources, you could also read in your own geometry here from a shapefile on disk.*

## 4. Intersecting Pixels w/ the Area of Interest

Now that we know how to get pixel data, and know how to get area of interest data, we can perform a spatial intersection to IDs of the pixels in the area of interest, which we'll use in a query to the Teragon API.

### With the ArcGIS API for Python

Using the `garrd_grid` feature layer and the `saw_mill_run` feature_set, running and intersect is pretty easy:

In [25]:
# construct the filter using the geometry module
sa_filter = geometry.filters.intersects(geometry=basin.features[0].geometry, sr=4326)
# then use that filter in a query of the the pixel data
pixels_of_interest = garrd_grid.query(geometry_filter=sa_filter, out_sr=4326)

In [26]:
m3 = gis.map('Pittsburgh')
m3.add_layer(pixels_of_interest)
m3

MapView(layout=Layout(height='400px', width='100%'))

There they are: pixels covering the Saw Mill Run watershed. Let's get a list of IDs, since that's what we're after.

First, let's introspect so we know what to go after:

In [27]:
pixels_of_interest.features[0]

{"geometry": {"rings": [[[-80.0100458361511, 40.5035909033969], [-80.0103477774924, 40.5125937398459], [-79.9985519610832, 40.5128236184692], [-79.9982515940814, 40.5038207517144], [-80.0100458361511, 40.5035909033969]]]}, "attributes": {"OBJECTID": 902, "PIXEL": "146131", "Shape__Area": 10763910.4165039, "Shape__Length": 13123.3595800102}}

We can see that each Feature object is represented as a Python dictionary, and the ID is stored under `attributes` in the `PIXEL` property. We can get all the Pixel IDS out into a list with a one-liner:

In [28]:
pixel_ids = list(set([f.attributes['PIXEL'] for f in pixels_of_interest.features]))
print(pixel_ids)

['146131', '147131', '146129', '147130', '146127', '149130', '146126', '145127', '149135', '150133', '146130', '148135', '148129', '145126', '144127', '149134', '143130', '145131', '149131', '143125', '148132', '143128', '148134', '148130', '143124', '149132', '147134', '149133', '147133', '145128', '150131', '147132', '145129', '145130', '144131', '144125', '144124', '143126', '144126', '144129', '150132', '148133', '145125', '146128', '143129', '144128', '148131', '143127', '144130']


### With GeoPandas

Alternatively, we can use the raw `geojson` that we've acquired in previous steps and find the spatial intersection using the GeoPandas library.

> *To be completed*

## 5. Half Circle: getting calibrated radar rainfall data (for an area of interest)

So far you've learned how to:

* make a request to Teragon Rainfall Dataset API
* get the Pixel reference geodata
* get area of interest reference data
* find the Pixels that are in the area of interest

Now it's time to bring it all together.

### First, though...

You'll recall that the Pixel list in the Teragon API looks something like this:

`'pixels': '148,134;149,134;148,133;149,133'`.

THAT is a semi-colon delimited list of Pixel IDs as a Python `str` object; each Pixel ID is split into two 3-digit IDs (which represents non-spatial grid coordinates). Our list of Pixel IDs does not look like that!

However, we can construct that exact thing from our list above with another one-liner.

In [29]:
pixel_ids_for_api = ";".join(["{0},{1}".format(i[:3], i[-3:]) for i in pixel_ids])
pixel_ids_for_api

'146,131;147,131;146,129;147,130;146,127;149,130;146,126;145,127;149,135;150,133;146,130;148,135;148,129;145,126;144,127;149,134;143,130;145,131;149,131;143,125;148,132;143,128;148,134;148,130;143,124;149,132;147,134;149,133;147,133;145,128;150,131;147,132;145,129;145,130;144,131;144,125;144,124;143,126;144,126;144,129;150,132;148,133;145,125;146,128;143,129;144,128;148,131;143,127;144,130'

Boom. A list of Pixel IDs in the format expected by the Teragon API.

### Make the Request

Let's use that in a request to the Teragon API, just like before

In [30]:
data = {
    'startyear': 2004,
    'startmonth': 9,
    'startday': 17,
    'starthour': 3,
    'endyear': 2004,
    'endmonth': 9,
    'endday': 18,
    'endhour': 0,
    'interval': 'Hourly',
    'zerofill': 'yes',
    'pixels': pixel_ids_for_api
}
rainfall_for_saw_mill_run = requests.post(
    url="http://web.3riverswetweather.org/trp:API.pixel",
    data=data
)

And a quick reformat and print (using the helper function we defined earlier):

In [31]:
rainfall_for_saw_mill_run_clean = clean_up_response(rainfall_for_saw_mill_run.text)

etl.vis.displayall(rainfall_for_saw_mill_run_clean)

Timestamp,146-131,147-131,146-129,147-130,146-127,149-130,146-126,145-127,149-135,150-133,146-130,148-135,148-129,145-126,144-127,149-134,143-130,145-131,149-131,143-125,148-132,143-128,148-134,148-130,143-124,149-132,147-134,149-133,147-133,145-128,150-131,147-132,145-129,145-130,144-131,144-125,144-124,143-126,144-126,144-129,150-132,148-133,145-125,146-128,143-129,144-128,148-131,143-127,144-130
2004-09-17T03:00:00,0.001,0.0,0.001,0.001,0.002,0.0,0.002,0.003,0.0,0.0,0.0,0.0,0.001,0.003,0.004,0.0,0.0,0.001,0.0,0.008,0.0,0.004,0.0,0.001,0.009,0.0,0.0,0.0,0.0,0.001,0.0,0.0,0.003,0.001,0.0,0.005,0.005,0.006,0.004,0.003,0.0,0.0,0.004,0.001,0.003,0.002,0.0,0.006,0.001
2004-09-17T04:00:00,0.004,0.007,0.02,0.019,0.025,0.029,0.029,0.024,0.003,0.007,0.016,0.005,0.03,0.029,0.025,0.001,0.003,0.004,0.019,0.033,0.008,0.012,0.004,0.022,0.033,0.007,0.008,0.002,0.003,0.014,0.023,0.002,0.02,0.007,0.003,0.032,0.032,0.033,0.03,0.009,0.014,0.005,0.031,0.021,0.002,0.01,0.014,0.025,0.003
2004-09-17T05:00:00,0.01,0.013,0.02,0.021,0.023,0.024,0.025,0.025,0.004,0.003,0.021,0.003,0.026,0.027,0.021,0.003,0.008,0.008,0.016,0.023,0.014,0.009,0.003,0.022,0.024,0.009,0.004,0.003,0.007,0.021,0.013,0.008,0.024,0.017,0.006,0.026,0.032,0.02,0.022,0.017,0.009,0.01,0.026,0.02,0.014,0.015,0.02,0.014,0.01
2004-09-17T06:00:00,0.029,0.028,0.022,0.025,0.015,0.031,0.015,0.016,0.037,0.04,0.026,0.043,0.026,0.014,0.019,0.04,0.024,0.029,0.029,0.017,0.031,0.019,0.043,0.026,0.015,0.037,0.04,0.041,0.036,0.016,0.032,0.031,0.022,0.027,0.022,0.014,0.015,0.018,0.016,0.023,0.035,0.045,0.014,0.016,0.023,0.016,0.029,0.019,0.024
2004-09-17T07:00:00,0.168,0.144,0.161,0.154,0.177,0.145,0.184,0.17,0.144,0.142,0.157,0.139,0.1549,0.183,0.175,0.1389,0.16,0.166,0.154,0.178,0.133,0.179,0.137,0.146,0.166,0.163,0.149,0.143,0.153,0.16,0.152,0.14,0.167,0.162,0.145,0.179,0.156,0.183,0.188,0.168,0.152,0.13,0.18,0.158,0.169,0.169,0.148,0.1829,0.169
2004-09-17T08:00:00,0.128,0.13,0.108,0.112,0.111,0.13,0.11,0.109,0.12,0.123,0.112,0.128,0.102,0.109,0.11,0.114,0.119,0.118,0.126,0.096,0.115,0.119,0.109,0.131,0.1,0.13,0.111,0.108,0.109,0.114,0.129,0.124,0.117,0.121,0.121,0.095,0.098,0.095,0.099,0.1219,0.118,0.104,0.095,0.107,0.128,0.118,0.133,0.106,0.12
2004-09-17T09:00:00,0.175,0.158,0.1469,0.152,0.152,0.157,0.145,0.158,0.215,0.203,0.163,0.233,0.143,0.151,0.154,0.22,0.163,0.185,0.186,0.144,0.165,0.1549,0.216,0.153,0.161,0.1799,0.21,0.2089,0.1879,0.143,0.194,0.1699,0.1419,0.176,0.1639,0.152,0.15,0.149,0.157,0.146,0.2029,0.177,0.162,0.1489,0.1629,0.144,0.161,0.15,0.1729
2004-09-17T10:00:00,0.38,0.36,0.371,0.389,0.412,0.3369,0.381,0.411,0.398,0.387,0.372,0.464,0.383,0.403,0.4249,0.3669,0.391,0.3799,0.369,0.3989,0.422,0.331,0.426,0.3619,0.3749,0.455,0.455,0.4109,0.426,0.451,0.375,0.395,0.364,0.342,0.386,0.357,0.353,0.386,0.416,0.371,0.422,0.461,0.3709,0.476,0.375,0.373,0.332,0.3979,0.3499
2004-09-17T11:00:00,0.518,0.472,0.442,0.453,0.405,0.335,0.4439,0.4308,0.337,0.393,0.426,0.362,0.363,0.46,0.456,0.41,0.4929,0.4779,0.425,0.397,0.468,0.472,0.452,0.408,0.4,0.515,0.5,0.456,0.5789,0.464,0.42,0.628,0.44,0.426,0.493,0.408,0.3569,0.4189,0.4399,0.446,0.4198,0.47,0.47,0.422,0.431,0.4939,0.397,0.471,0.4869
2004-09-17T12:00:00,0.4429,0.411,0.309,0.3399,0.3309,0.309,0.422,0.391,0.727,0.463,0.348,0.597,0.281,0.418,0.401,0.629,0.42,0.46,0.338,0.381,0.4009,0.403,0.559,0.3189,0.534,0.404,0.4979,0.476,0.4668,0.348,0.365,0.401,0.3519,0.378,0.483,0.37,0.388,0.358,0.396,0.388,0.401,0.439,0.405,0.3159,0.405,0.3799,0.342,0.394,0.405


## 6. Full Circle: putting a summary of that calibrated radar rainfall data for an area of interest on a map

In [32]:
from statistics import mean, stdev

In [41]:
rainfall_smr_totals = etl\
    .transpose(rainfall_for_saw_mill_run_clean)\
    .aggregate(
        'Timestamp', 
        {
            'sum': (list(etl.values(rainfall_for_saw_mill_run_clean, 'Timestamp')), list),
            'mean': (list(etl.values(rainfall_for_saw_mill_run_clean, 'Timestamp')), list),
            'stdev': (list(etl.values(rainfall_for_saw_mill_run_clean, 'Timestamp')), list),
            'min': (list(etl.values(rainfall_for_saw_mill_run_clean, 'Timestamp')), list),
            'max': (list(etl.values(rainfall_for_saw_mill_run_clean, 'Timestamp')), list)
        }
    )\
    .convert('sum', lambda v: round(sum(list(v[0])), 3))\
    .convert('mean', lambda v: round(mean(list(v[0])), 3))\
    .convert('stdev', lambda v: round(stdev(list(v[0])), 3))\
    .convert('min', lambda v: round(min(list(v[0])), 3))\
    .convert('max', lambda v: round(max(list(v[0])), 3))\
    .rename('Timestamp', 'PIXEL')\
    .convert('PIXEL', lambda v: str("".join(v.split("-"))))
    

etl.vis.display(rainfall_smr_totals)

PIXEL,sum,mean,stdev,min,max
143124,6.453,0.307,0.323,0.009,1.218
143125,6.356,0.303,0.302,0.008,1.127
143126,6.551,0.312,0.323,0.006,1.179
143127,6.675,0.318,0.329,0.006,1.145
143128,6.749,0.321,0.339,0.004,1.175


In [35]:
pixels_geojson = json.loads(pixels_of_interest.to_geojson)
pixels_geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-80.0100458361511, 40.5035909033969],
      [-80.0103477774924, 40.5125937398459],
      [-79.9985519610832, 40.5128236184692],
      [-79.9982515940814, 40.5038207517144],
      [-80.0100458361511, 40.5035909033969]]]},
   'properties': {'OBJECTID': 902,
    'PIXEL': '146131',
    'Shape__Area': 10763910.4165039,
    'Shape__Length': 13123.3595800102}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-80.0345495922899, 40.5301358763873],
      [-80.0348549268468, 40.5391386019192],
      [-80.0230545468563, 40.5393709774553],
      [-80.0227507878347, 40.5303682212934],
      [-80.0345495922899, 40.5301358763873]]]},
   'properties': {'OBJECTID': 733,
    'PIXEL': '144128',
    'Shape__Area': 10763910.416626,
    'Shape__Length': 13123.3595799804}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-79.9625

In [36]:
new_fs = []
# for f in pixels_geojson['features']:
for f in pixels_of_interest.features:
    #p = f['properties']['PIXEL']
    p = f.attributes['PIXEL']
    t = etl\
        .selecteq(rainfall_smr_totals, 'PIXEL', p)\
        .cutout('PIXEL')\
        .dicts()
    #print(p, t[0])
    #f['properties'].update(t[0])
    f.attributes.update(t[0])

In [37]:
new_fields = [
    {
        'name': f,
        'type': 'esriFieldTypeDouble',
        'alias': f,
        'sqlType': 'sqlTypeOther',
        'domain': None,
        'defaultValue': None
    } for f in [h for h in list(etl.header(rainfall_smr_totals)) if h != "PIXEL"]
]

pixels_of_interest.fields.extend(new_fields)

In [42]:
options={"opacity":1, "renderer": "ClassedColorRenderer", "field_name":"sum"}

In [43]:
map_widget = gis.map('Pittsburgh')
map_widget.add_layer(pixels_of_interest, options=options)
map_widget

MapView(layout=Layout(height='400px', width='100%'))